# Task 3 : OpenHygrisC Nitrate Data 

## importing the messwert Table to the Postgres Database

The aim of this Notebook is to import the Messwert Table which is provided by LANUV to our postgres Database.

first we need follwing pachakges:
* Pandas 
    
    Pandas is essentioal to read and manage our data in the data frame.
* sqlalchemy:
 
 sqlalchemy is essentioal to read from database, write to database, connect to database and so on.


In [3]:
import pandas as pd
import sqlalchemy
import credential_temp as  creds 

## read the Data

The name of our CSV file is "opendata_messwert" which is avaliable in data folder. 

In [4]:
import pandas as pd

# Define the list of CSV files
csv_files = ["data/OpenHygrisC_gw-chemischer-messwert_1922-1969.csv", "data/OpenHygrisC_gw-chemischer-messwert_1970-1989.csv", "data/OpenHygrisC_gw-chemischer-messwert_1990-1999.csv", "data/OpenHygrisC_gw-chemischer-messwert_2000-2009.csv", "data/OpenHygrisC_gw-chemischer-messwert_2010-2019.csv", "data/OpenHygrisC_gw-chemischer-messwert_2020-2029.csv"]

# Initialize an empty list to hold column sets
columns_list = []

# Iterate over the list of files and check their columns
for file in csv_files:
    df = pd.read_csv(file, nrows=0 ,encoding='ISO-8859-1')  # Only read the header to get the columns
    columns_list.append(set(df.columns))

# Compare all columns sets to see if they are the same
all_columns_same = all(columns == columns_list[0] for columns in columns_list)

if all_columns_same:
    print("All CSV files have the same columns.")
else:
    print("CSV files have different columns.")
    
    # Optionally, display the differences
    for i, columns in enumerate(columns_list):
        print(f"Columns in {csv_files[i]}: {columns}")


All CSV files have the same columns.


In [5]:
import pandas as pd

# List of CSV files to read
csv_files = ["data/OpenHygrisC_gw-chemischer-messwert_1922-1969.csv", "data/OpenHygrisC_gw-chemischer-messwert_1970-1989.csv", "data/OpenHygrisC_gw-chemischer-messwert_1990-1999.csv", "data/OpenHygrisC_gw-chemischer-messwert_2000-2009.csv", "data/OpenHygrisC_gw-chemischer-messwert_2010-2019.csv", "data/OpenHygrisC_gw-chemischer-messwert_2020-2029.csv"]

# Initialize an empty list to store DataFrames
dfs = []

# Loop over each file, read it, and append to the list
for file in csv_files:
    df = pd.read_csv(file, sep=";", index_col=["sl_nr"], dtype={"messergebnis_c": str, "messergebnis_hinweis": str}, encoding='ISO-8859-1')
    dfs.append(df)
    # Print the columns to verify
    print(f"Columns in {file}: {df.columns.tolist()}")

# Concatenate all DataFrames
df_qual = pd.concat(dfs, axis=0)

# Display the combined DataFrame
print(df_qual.head())

Columns in data/OpenHygrisC_gw-chemischer-messwert_1922-1969.csv: ['messstelle_id', 'messstelle_sl_nr', 'datum_pn', 'stoff_nr', 'probengut', 'messergebnis_c', 'messergebnis_hinweis', 'bestimmungsgrenze', 'masseinheit', 'trennverfahren', 'verfahren', 'vor_ort']
Columns in data/OpenHygrisC_gw-chemischer-messwert_1970-1989.csv: ['messstelle_id', 'messstelle_sl_nr', 'datum_pn', 'stoff_nr', 'probengut', 'messergebnis_c', 'messergebnis_hinweis', 'bestimmungsgrenze', 'masseinheit', 'trennverfahren', 'verfahren', 'vor_ort']
Columns in data/OpenHygrisC_gw-chemischer-messwert_1990-1999.csv: ['messstelle_id', 'messstelle_sl_nr', 'datum_pn', 'stoff_nr', 'probengut', 'messergebnis_c', 'messergebnis_hinweis', 'bestimmungsgrenze', 'masseinheit', 'trennverfahren', 'verfahren', 'vor_ort']
Columns in data/OpenHygrisC_gw-chemischer-messwert_2000-2009.csv: ['messstelle_id', 'messstelle_sl_nr', 'datum_pn', 'stoff_nr', 'probengut', 'messergebnis_c', 'messergebnis_hinweis', 'bestimmungsgrenze', 'masseinheit'

In [6]:
#data_in_dir = r"data/"

#gw_quality_fname = r"OpenHygrisC_gw-chemischer-messwert.csv"

#gw_quality_pfname = data_in_dir + gw_quality_fname

Now we need to read our csv files in the pandas data fram
##### Don't forget to add the "delimiter", normal csv file's delimiter is "," but in our case it's ";", so we need to assign it.

Also in the same time, I have assigned the "str" type to the two columns. and because "sl nr" is unique value then I decided to choose it as an index.


In [7]:
#df_qual = pd.read_csv(gw_quality_pfname, sep = ";", index_col=["sl_nr"], dtype = {"messergebnis_c":str ,"messergebnis_hinweis":str },encoding='ISO-8859-1')
#df_qual.head()

## changing the Data type
we need to change some columns's data type. 

 we have three date columns but they are not in Date type, they are just simply object which means string.

In [8]:
df_qual.dtypes

messstelle_id            int64
messstelle_sl_nr         int64
datum_pn                object
stoff_nr                 int64
probengut               object
messergebnis_c          object
messergebnis_hinweis    object
bestimmungsgrenze       object
masseinheit             object
trennverfahren          object
verfahren               object
vor_ort                 object
dtype: object

In [9]:
df_qual['datum_pn'] = pd.to_datetime(df_qual['datum_pn'], format='%Y-%m-%d')

In [10]:
df_qual.dtypes

messstelle_id                    int64
messstelle_sl_nr                 int64
datum_pn                datetime64[ns]
stoff_nr                         int64
probengut                       object
messergebnis_c                  object
messergebnis_hinweis            object
bestimmungsgrenze               object
masseinheit                     object
trennverfahren                  object
verfahren                       object
vor_ort                         object
dtype: object

* Now all the tree columns which were date are in datetime64 type

### Create functions

Now the most important columns is "messergebnis_c" which simply means the result of the measurments. The data type of this columns should be float. But we have probelm. some value in "messergebnis_c" starts with < or > and that's why the pandas automatically assign object data type to this column.  First we need to create some functions to check that how many "messergebnis_c" consist of < and how many consist of > and how many value of "messergebnis_c" are with no greater than and less than sign. 

* First function: 

 This function is to check that are we able to convert the string to float?

In [11]:
# check if string can be converted to float
def is_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

* Second function: 

The function is to check that the str starts with '<' 

In [12]:
# check if string starts with '<'
def is_less(element: str) -> bool:
    return element[0] == "<" 

* Third function:

The function is to check that the str starts with '>'

In [13]:
# check if string starts with '>'
def is_greater(element: str) -> bool:
    return element[0] == ">" 

### Now apply all the three above functions on the "messergebnis_c" columns

In [14]:
%time idx_mess_is_float   = df_qual["messergebnis_c"].apply(is_float)
#%time idx_mess_not_float  = ~idx_mess_is_float
%time idx_mess_is_less    = df_qual["messergebnis_c"].apply(is_less)
%time idx_mess_is_greater = df_qual["messergebnis_c"].apply(is_greater)

CPU times: total: 5.22 s
Wall time: 5.65 s
CPU times: total: 1.12 s
Wall time: 1.14 s
CPU times: total: 1.14 s
Wall time: 1.15 s


### Now we can see that "messergebnis_c" of how many rows are with "<" and ">" or with no sign which means they can simply convert to float with no problem.

In [15]:
# How many can be converted to float with no problem
df_qual[idx_mess_is_float].shape

(0, 12)

In [16]:
# How many consist of < 
df_qual[idx_mess_is_less].shape

(2763564, 12)

In [17]:
# how many consist of >
df_qual[idx_mess_is_greater].shape

(32, 12)

## check again

Now is the time to check again all the value of "messergebnis_c" to be sure that there is no other probelm

In [18]:
# Recalculate the boolean masks
idx_mess_is_less = df_qual['messergebnis_c'].str.contains('<', regex=False)
idx_mess_is_greater = df_qual['messergebnis_c'].str.contains('>', regex=False)
idx_mess_is_float = df_qual['messergebnis_c'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull()


In [19]:
# Identify rows that are causing the issue
problematic_rows = df_qual[(~idx_mess_is_float) & ~(idx_mess_is_less | idx_mess_is_greater)]

# Display the problematic rows
print(problematic_rows[['messergebnis_c']])


         messergebnis_c
sl_nr                  
4018042         3,50000
4018043         6,80000
4018044        85,00000
4018045         0,23256
4018046         0,03049
...                 ...
20639773        0,28000
20639295      702,00000
20639275        0,00008
20920714        0,72000
20640541        6,50000

[2343618 rows x 1 columns]


In [20]:
df_qual['messergebnis_c'] = df_qual['messergebnis_c'].str.strip()
df_qual['messergebnis_c'] = df_qual['messergebnis_c'].str.replace(r'[^\x20-\x7E]', '', regex=True)


In [21]:
unique_values = df_qual['messergebnis_c'].unique()
print(unique_values)


['<0,10000' '<0,02200' '<0,05000' ... '2.393,00000' '1.348,00000'
 '2.424,00000']


In [22]:
# Remove all characters except digits, decimal points, '<', '>'
df_qual['messergebnis_c'] = df_qual['messergebnis_c'].str.replace(r'[^\d.<>=]', '', regex=True)


In [23]:
df_qual['messergebnis_c'] = df_qual['messergebnis_c'].str.replace(',', '.', regex=False)

In [24]:
idx_mess_is_less = df_qual['messergebnis_c'].str.contains('<', regex=False)
idx_mess_is_greater = df_qual['messergebnis_c'].str.contains('>', regex=False)
idx_mess_is_float = df_qual['messergebnis_c'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull()

In [25]:
assert idx_mess_is_float.sum() + (~idx_mess_is_float).sum() == df_qual.shape[0], "mess has either to be float or not float"

In [26]:
# ((idx_mess_is_less | idx_mess_is_greater) == idx_mess_not_float).value_counts()
assert ((idx_mess_is_less | idx_mess_is_greater) != (~idx_mess_is_float)).sum() == 0, "'not float' supposed to come from '<' or '>' but nothing else."

In [27]:
# Print records which are neither less nor greater nor float -> should be empty data frame
assert df_qual[~idx_mess_is_less & ~idx_mess_is_greater & ~idx_mess_is_float].shape[0] == 0
print(df_qual[~idx_mess_is_less & ~idx_mess_is_greater & ~idx_mess_is_float])

Empty DataFrame
Columns: [messstelle_id, messstelle_sl_nr, datum_pn, stoff_nr, probengut, messergebnis_c, messergebnis_hinweis, bestimmungsgrenze, masseinheit, trennverfahren, verfahren, vor_ort]
Index: []


## Create a new column

In this step, we want to create a new column with the name of "messergebnis_cm". The value in this column is based on "messergebnis_c". There are three diffrent values in this column.

* = : Which means that the value in messergebnis_c has no sign (< or >)
* > : Which means that the value in messergebnis_c starts with sign of >
* < : Which means that the value in messergebnis_c starts with sogn of <

In [28]:
df_qual.loc[idx_mess_is_float, "messergebnis_cm"] = "="
df_qual.loc[idx_mess_is_less, "messergebnis_cm"] = "<"
df_qual.loc[idx_mess_is_greater, "messergebnis_cm"] = ">"

## replace the signs with noting

Now is the time to delete the < and > from the biggining of each cell. but acctually we have these value in another column (messergebnis_cm)

In [29]:
df_qual.messergebnis_c = df_qual.messergebnis_c.replace({'>':''}, regex=True)

In [30]:
df_qual.messergebnis_c = df_qual.messergebnis_c.replace({'<':''}, regex=True)

## check the datafram again

In [31]:
df_qual

,messstelle_id,messstelle_sl_nr,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,messergebnis_cm
sl_nr,,,,,,,,,,,,,
9753769,21172109,3528,1965-04-27,1171,Grundwasser,010000,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN 38406-E2 MAI 1983,,<
4018047,42018808,10612,1966-04-12,1264,Grundwasser,002200,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 1189 (1996),,<
4018053,42018808,10612,1966-04-12,1561,Grundwasser,005000,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 903 (1994),,<
4018050,42018808,10612,1966-04-12,1182,Grundwasser,005000,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN ISO 11885 (1998),,<
13130505,42018808,10612,1966-04-12,1263,Grundwasser,006769,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 1189 (1996),,<
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20639785,60220909,13063,2022-06-30,4393,Grundwasser,005000,Konzentration zu gering zur Bestimmung ...,"0,05000",µg/l,Gesamtgehalt,"Direkt, LC-MS-MS - VS - FY0 - a",,<
20640175,60220909,13063,2022-06-30,4099,Grundwasser,005000,Konzentration zu gering zur Bestimmung ...,"0,05000",µg/l,Gesamtgehalt,"Direkt, LC-MS-MS - VS - FY0 - a",,<
20640193,60220909,13063,2022-06-30,4210,Grundwasser,002500,Konzentration zu gering zur Bestimmung ...,"0,02500",µg/l,Gesamtgehalt,DIN EN ISO 21676 - DO - F47 -1,,<


## converting messergebnis_c to float

Finally we are in the stage that we can now convert the messergebnis_c to float

In [32]:
 df_qual.messergebnis_c = df_qual.messergebnis_c.astype(float)

In [33]:
df_qual.dtypes

messstelle_id                    int64
messstelle_sl_nr                 int64
datum_pn                datetime64[ns]
stoff_nr                         int64
probengut                       object
messergebnis_c                 float64
messergebnis_hinweis            object
bestimmungsgrenze               object
masseinheit                     object
trennverfahren                  object
verfahren                       object
vor_ort                         object
messergebnis_cm                 object
dtype: object

## Eerything is fine now

As you can simply see above that all the columns's type is what we want. 

* messergebnis_c is now in float
* datum_pn is now in datetime64


## once again check externally to be 100% sure

In [34]:
# # write on csv to check externally
# df_qual.to_csv("test.csv")

## Import to Database

Now is the time to import our datafram to database with no problem. 

First we need to connect  to our database. and then import the datafram. 

In this case, I have imported the data to schema "hygrisc" which has been already created in the PGadmin by myself, but you can import to your Schema.

In [35]:
# connect to out credential file

postgresurl = creds.URL


In [36]:
# replace your database link in below
engine = sqlalchemy.create_engine(postgresurl)

In [37]:
chunk_size = 50000  # Adjust chunk size based on your system's memory
for start in range(0, len(df_qual), chunk_size):
    end = min(start + chunk_size, len(df_qual))
    df_chunk = df_qual.iloc[start:end]
    df_chunk.to_sql(con=engine, name="messwert", schema="hygrisc", if_exists="append", index=False)


In [ ]:
#%time df_qual.to_sql(con=engine, name="messwert", schema="hygrisc", if_exists="replace", index=False)

In [38]:
df_qual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5107214 entries, 9753769 to 20640545
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   messstelle_id         int64         
 1   messstelle_sl_nr      int64         
 2   datum_pn              datetime64[ns]
 3   stoff_nr              int64         
 4   probengut             object        
 5   messergebnis_c        float64       
 6   messergebnis_hinweis  object        
 7   bestimmungsgrenze     object        
 8   masseinheit           object        
 9   trennverfahren        object        
 10  verfahren             object        
 11  vor_ort               object        
 12  messergebnis_cm       object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 545.5+ MB


In [39]:
df_qual.head()

,messstelle_id,messstelle_sl_nr,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,messergebnis_cm
sl_nr,,,,,,,,,,,,,
9753769,21172109,3528,1965-04-27,1171,Grundwasser,10000.0,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN 38406-E2 MAI 1983,,<
4018047,42018808,10612,1966-04-12,1264,Grundwasser,2200.0,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 1189 (1996),,<
4018053,42018808,10612,1966-04-12,1561,Grundwasser,5000.0,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 903 (1994),,<
4018050,42018808,10612,1966-04-12,1182,Grundwasser,5000.0,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN ISO 11885 (1998),,<
13130505,42018808,10612,1966-04-12,1263,Grundwasser,6769.0,Konzentration zu gering zur Bestimmung ...,NaN,mg/l,Gesamtgehalt,DIN EN 1189 (1996),,<


# Finish !
## Good Luck, You are doing well

In [87]:
# SQL command to drop the table
drop_table_query = "DROP TABLE IF EXISTS messwert;"

try:
    # Execute the command to drop the table
    cursor.execute(drop_table_query)
    conn.commit()
    print(f"Table 'messwert' deleted successfully.")

except Exception as error:
    print(f"Error: {error}")

Error: name 'cursor' is not defined
